In [1]:
# coding='utf-8'
import os
import sys
import numpy as np
import time
import datetime
import json
import importlib
import logging
import shutil

import torch
import torch.nn as nn

from yolo_model import yoloModel
from PASCAL_Dataloader import create_split_loaders
from YOLO_Loss import YoloLoss
from utils import NMS
from bbox import bbox_iou

In [2]:
def evaluate(config):
    # Load and initialize network
    net = yoloModel(config)

    # Set data parallel
    net = net.cuda()

    # Restore pretrain model
    if config["pretrain_snapshot"]:
        state_dict = torch.load(config["pretrain_snapshot"])
        net.load_state_dict(state_dict)
    else:
        logging.warning("missing pretrain_snapshot!!!")

    # YOLO loss with 3 scales
    yolo_losses = []
    for i in range(3):
        yolo_losses.append(YoloLoss(config["classes"], (config["img_w"], config["img_h"]), config["anchors"][i]))

    # DataLoader
    root_dir = os.getcwd()
    train_loader, val_loader, test_loader = create_split_loaders(root_dir, config['batch_size'])

    # Start the eval loop
    logging.info("Start eval.")
    n_gt = 0
    correct = 0
    for step, samples in enumerate(test_loader):
        images, labels = samples["image"], samples["label"]
        images,labels = images.cuda(),labels.cuda()
        with torch.no_grad():
            outputs = net(images)
            output_list = []
            for i in range(3):
                output_list.append(yolo_losses[i](outputs[i]))
            output = torch.cat(output_list, 1)
            output = NMS(output, config["classes"], conf_thresh=0.2)
            #  calculate
            for sample_i in range(labels.size(0)):
                # Get labels for sample where width is not zero (dummies)
                target_sample = labels[sample_i, labels[sample_i, :, 3] != 0]
                for obj_cls, tx, ty, tw, th in target_sample:
                    # Get rescaled gt coordinates
                    tx1, tx2 = config["img_w"] * (tx - tw / 2), config["img_w"] * (tx + tw / 2)
                    ty1, ty2 = config["img_h"] * (ty - th / 2), config["img_h"] * (ty + th / 2)
                    n_gt += 1
                    box_gt = torch.cat([coord.unsqueeze(0) for coord in [tx1, ty1, tx2, ty2]]).view(1, -1)
                    box_gt = box_gt.float()
                    sample_pred = output[sample_i]
                    print(sample_pred)
                    print(obj_cls)
                    if sample_pred is not None:
                        # Iterate through predictions where the class predicted is same as gt
                        for x1, y1, x2, y2, conf, obj_conf, obj_pred in sample_pred[sample_pred[:, 6] == obj_cls.float()]:
                            box_pred = torch.cat([coord.unsqueeze(0) for coord in [x1, y1, x2, y2]]).view(1, -1)
                            iou = bbox_iou(box_pred, box_gt)
                            print(iou)
                            if iou >= config["confidence_threshold"]:
                                correct += 1
                                break
        if n_gt:
            logging.info('Batch [%d/%d] mAP: %.5f' % (step, len(test_loader), float(correct / n_gt)))

    logging.info('Mean Average Precision: %.5f' % float(correct / n_gt))

In [3]:
def main():
    logging.basicConfig(level=logging.DEBUG,
                        format="[%(asctime)s %(filename)s] %(message)s")

    config = {}
    config["batch_size"] = 1
    config['backbone_name'] = "darknet_53"
    config['backbone_pretrained'] = ""
    config['anchors'] = [[[116, 90], [156, 198], [373, 326]],
                                [[30, 61], [62, 45], [59, 119]],
                                [[10, 13], [16, 30], [33, 23]]]
    config['classes'] = 20
    config['img_h'] = config['img_w'] = 416
    config['confidence_threshold'] = 0.5
    config['pretrain_snapshot'] = "./states/20190530140657/model.pth"
    config['classes_names_path'] = "./data/voc.names"
    config['images_path'] = "./images"

    # Start training
    evaluate(config)

if __name__ == "__main__":
    main()

[2019-05-30 22:19:14,291 <ipython-input-2-1b59063d1b23>] Start eval.
[2019-05-30 22:19:16,819 <ipython-input-2-1b59063d1b23>] Batch [0/2077] mAP: 0.00000
[2019-05-30 22:19:16,875 <ipython-input-2-1b59063d1b23>] Batch [1/2077] mAP: 0.00000
[2019-05-30 22:19:16,906 <ipython-input-2-1b59063d1b23>] Batch [2/2077] mAP: 0.00000
[2019-05-30 22:19:16,942 <ipython-input-2-1b59063d1b23>] Batch [3/2077] mAP: 0.00000
[2019-05-30 22:19:16,975 <ipython-input-2-1b59063d1b23>] Batch [4/2077] mAP: 0.00000


tensor([[2.1343e+02, 2.1351e+02, 4.4259e+02, 3.5448e+02, 7.4389e-01, 4.9244e-02,
         2.0000e+00],
        [2.0741e+02, 2.3550e+02, 3.0423e+02, 3.9708e+02, 8.3398e-01, 4.0853e-02,
         1.1000e+01],
        [2.3306e+02, 2.1285e+02, 5.2631e+02, 4.3697e+02, 6.6868e-01, 7.2369e-02,
         1.2000e+01],
        [2.3052e+02, 2.3452e+02, 2.9840e+02, 3.1191e+02, 6.8049e-01, 5.4776e-02,
         1.4000e+01]], device='cuda:0')
tensor(12., device='cuda:0', dtype=torch.float64)
tensor([0.1705], device='cuda:0')
tensor([[2.0698e+02, 2.0727e+02, 3.8254e+02, 1.9378e+02, 2.9519e-01, 4.7691e-01,
         0.0000e+00],
        [1.1444e+02, 1.7654e+02, 1.8410e+02, 1.6261e+02, 2.0869e-01, 1.4781e-01,
         8.0000e+00],
        [2.0876e+01, 3.0402e+02, 1.8524e+01, 7.5998e+01, 5.3137e-01, 9.5739e-01,
         1.4000e+01]], device='cuda:0')
tensor(0., device='cuda:0', dtype=torch.float64)
tensor([0.], device='cuda:0')
tensor([[234.2069, 208.5324, 205.7854, 313.1804,   0.8646,   0.9204,  14.0000]],

[2019-05-30 22:19:17,015 <ipython-input-2-1b59063d1b23>] Batch [5/2077] mAP: 0.00000
[2019-05-30 22:19:17,076 <ipython-input-2-1b59063d1b23>] Batch [6/2077] mAP: 0.00000
[2019-05-30 22:19:17,119 <ipython-input-2-1b59063d1b23>] Batch [7/2077] mAP: 0.00000
[2019-05-30 22:19:17,172 <ipython-input-2-1b59063d1b23>] Batch [8/2077] mAP: 0.00000
[2019-05-30 22:19:17,206 <ipython-input-2-1b59063d1b23>] Batch [9/2077] mAP: 0.00000
[2019-05-30 22:19:17,239 <ipython-input-2-1b59063d1b23>] Batch [10/2077] mAP: 0.00000


tensor([[2.0523e+02, 2.1580e+02, 4.9139e+02, 2.2482e+02, 7.8982e-01, 8.4113e-02,
         5.0000e+00],
        [2.0511e+02, 2.3417e+02, 5.9030e+02, 3.8378e+02, 7.9530e-01, 1.5972e-01,
         6.0000e+00],
        [1.5162e+02, 2.3497e+02, 2.4562e+02, 2.1155e+02, 2.6748e-01, 9.0311e-02,
         1.4000e+01]], device='cuda:0')
tensor(6., device='cuda:0', dtype=torch.float64)
tensor([0.2009], device='cuda:0')
tensor([[2.0912e+02, 2.3767e+02, 1.5683e+02, 2.6010e+02, 4.6915e-01, 2.5257e-01,
         1.4000e+01],
        [2.1031e+02, 8.0545e+01, 8.6006e+01, 8.8616e+01, 3.3736e-01, 7.8498e-01,
         1.9000e+01]], device='cuda:0')
tensor(19., device='cuda:0', dtype=torch.float64)
tensor([0.], device='cuda:0')
tensor([[2.0912e+02, 2.3767e+02, 1.5683e+02, 2.6010e+02, 4.6915e-01, 2.5257e-01,
         1.4000e+01],
        [2.1031e+02, 8.0545e+01, 8.6006e+01, 8.8616e+01, 3.3736e-01, 7.8498e-01,
         1.9000e+01]], device='cuda:0')
tensor(4., device='cuda:0', dtype=torch.float64)
tensor([[2.09

[2019-05-30 22:19:17,280 <ipython-input-2-1b59063d1b23>] Batch [11/2077] mAP: 0.00000
[2019-05-30 22:19:17,314 <ipython-input-2-1b59063d1b23>] Batch [12/2077] mAP: 0.00000
[2019-05-30 22:19:17,352 <ipython-input-2-1b59063d1b23>] Batch [13/2077] mAP: 0.00000
[2019-05-30 22:19:17,387 <ipython-input-2-1b59063d1b23>] Batch [14/2077] mAP: 0.00000
[2019-05-30 22:19:17,451 <ipython-input-2-1b59063d1b23>] Batch [15/2077] mAP: 0.00000


...

[2019-05-30 22:20:50,732 <ipython-input-2-1b59063d1b23>] Batch [2070/2077] mAP: 0.00000
[2019-05-30 22:20:50,783 <ipython-input-2-1b59063d1b23>] Batch [2071/2077] mAP: 0.00000
[2019-05-30 22:20:50,825 <ipython-input-2-1b59063d1b23>] Batch [2072/2077] mAP: 0.00000
[2019-05-30 22:20:50,877 <ipython-input-2-1b59063d1b23>] Batch [2073/2077] mAP: 0.00000


tensor([[8.0576e+01, 1.7817e+02, 1.2008e+02, 1.2213e+02, 5.8988e-01, 5.6386e-01,
         6.0000e+00],
        [8.1831e+01, 1.9880e+02, 8.0519e+01, 1.0216e+02, 2.0301e-01, 2.1803e-01,
         8.0000e+00],
        [3.7244e+02, 2.3844e+02, 6.8728e+01, 1.5439e+02, 5.5616e-01, 5.6822e-01,
         1.4000e+01]], device='cuda:0')
tensor(14., device='cuda:0', dtype=torch.float64)
tensor([0.], device='cuda:0')
tensor([[8.0576e+01, 1.7817e+02, 1.2008e+02, 1.2213e+02, 5.8988e-01, 5.6386e-01,
         6.0000e+00],
        [8.1831e+01, 1.9880e+02, 8.0519e+01, 1.0216e+02, 2.0301e-01, 2.1803e-01,
         8.0000e+00],
        [3.7244e+02, 2.3844e+02, 6.8728e+01, 1.5439e+02, 5.5616e-01, 5.6822e-01,
         1.4000e+01]], device='cuda:0')
tensor(14., device='cuda:0', dtype=torch.float64)
tensor([0.], device='cuda:0')
tensor([[8.0576e+01, 1.7817e+02, 1.2008e+02, 1.2213e+02, 5.8988e-01, 5.6386e-01,
         6.0000e+00],
        [8.1831e+01, 1.9880e+02, 8.0519e+01, 1.0216e+02, 2.0301e-01, 2.1803e-01,
  

[2019-05-30 22:20:50,925 <ipython-input-2-1b59063d1b23>] Batch [2074/2077] mAP: 0.00000
[2019-05-30 22:20:50,976 <ipython-input-2-1b59063d1b23>] Batch [2075/2077] mAP: 0.00000
[2019-05-30 22:20:51,010 <ipython-input-2-1b59063d1b23>] Batch [2076/2077] mAP: 0.00000


tensor([[2.3983e+02, 4.3665e+01, 5.2611e+01, 5.5003e+01, 3.7807e-01, 4.4790e-01,
         1.0000e+00],
        [1.7087e+02, 2.4141e+02, 3.5831e+02, 2.7180e+02, 7.6497e-01, 2.1481e-01,
         5.0000e+00]], device='cuda:0')
tensor(5., device='cuda:0', dtype=torch.float64)
tensor([0.0552], device='cuda:0')
tensor([[2.3983e+02, 4.3665e+01, 5.2611e+01, 5.5003e+01, 3.7807e-01, 4.4790e-01,
         1.0000e+00],
        [1.7087e+02, 2.4141e+02, 3.5831e+02, 2.7180e+02, 7.6497e-01, 2.1481e-01,
         5.0000e+00]], device='cuda:0')
tensor(14., device='cuda:0', dtype=torch.float64)
tensor([[2.3983e+02, 4.3665e+01, 5.2611e+01, 5.5003e+01, 3.7807e-01, 4.4790e-01,
         1.0000e+00],
        [1.7087e+02, 2.4141e+02, 3.5831e+02, 2.7180e+02, 7.6497e-01, 2.1481e-01,
         5.0000e+00]], device='cuda:0')
tensor(1., device='cuda:0', dtype=torch.float64)
tensor([-0.], device='cuda:0')
tensor([[2.3983e+02, 4.3665e+01, 5.2611e+01, 5.5003e+01, 3.7807e-01, 4.4790e-01,
         1.0000e+00],
        [1.7

[2019-05-30 22:20:51,646 <ipython-input-2-1b59063d1b23>] Mean Average Precision: 0.00000
